<a href="https://colab.research.google.com/github/melissatorgbi/LLMCxG_Workshop/blob/main/Live_LLMCxG_Notebook_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This notebook uses data from the paper ["Assessing Language Comprehension in Large Language Models Using Construction Grammar"](https://arxiv.org/abs/2501.04661).

Here we focus on a subset of the data that is based on the Natural Language Inference (NLI) task and construction grammar.

We use the OpenAI API to prompt a model so an API key is required.

## Natural Language Inference (NLI)

NLI involves determining the inference relation between two short texts (Premise and the Hypothesis). The possible relations are entailment, contradiction, or neutral.

0 – entailment – The hypothesis must be true given the premise

1 – neutral – The hypothesis may or may not be true given the premise

2 – contradiction – The hypothesis must not be true given the premise

**Examples:**

Premise: Children smiling and waving at camera

Hypothesis: They are smiling at their parents

Relation: 1 neutral

Premise: Children smiling and waving at camera

Hypothesis: There are children present

Relation: 0 entailment





Premise: Children smiling and waving at camera

Hypothesis: The kids are frowning

Relation: 2 contradiction




## Imports

In [ ]:
!git clone https://github.com/melissatorgbi/Assessing-Language-Comprehension-in-Large-Language-Models-Using-Construction-Grammar.git
%cd Assessing-Language-Comprehension-in-Large-Language-Models-Using-Construction-Grammar

Cloning into 'Assessing-Language-Comprehension-in-Large-Language-Models-Using-Construction-Grammar'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 68 (delta 20), reused 46 (delta 14), pack-reused 0 (from 0)
Receiving objects: 100% (68/68), 83.41 KiB | 1.10 MiB/s, done.
Resolving deltas: 100% (20/20), done.
/content/Assessing-Language-Comprehension-in-Large-Language-Models-Using-Construction-Grammar/Assessing-Language-Comprehension-in-Large-Language-Models-Using-Construction-Grammar


In [ ]:
import pandas as pd
import csv
import os
import re
from openai import OpenAI

# Set-up

In [ ]:
OPENAI_API_KEY = "replace the text with your API key" #replace the text with your API key

In [ ]:
client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
# function to load data from csv

def load_csv( file_location ) :
      header = None
      data   = list()
      path, extension = os.path.splitext(file_location)
      with open( file_location ) as csvfile :
          if extension == ".csv":
              reader = csv.reader(csvfile)
          elif extension == ".tsv":
              reader = csv.reader(csvfile, delimiter="\t")
          for row in reader:
              if header is None:
                  header = row
                  continue
              data.append( row )
      return data

In [ ]:
# function to change format of the data

def load_data(file_location):

        data = load_csv(file_location)

        out_data = list()

        for i in range( 0, len(data), 3 ) :

            this_cxg = data[i][0]
            assert this_cxg != ''
            assert data[i][2] in [ 'Premi', 'premise' ]
            this_premise = data[i][3]
            assert data[i+1][2] == 'hypothesis'
            this_hypothesis = data[i+1][3]
            assert data[i+2][2] == 'relation'
            this_relation   = data[i+2][3]

            this_relation   = int( re.sub( r'\(.*\)', '', this_relation ) )
            this_trip = [this_cxg, this_premise,this_hypothesis, this_relation]
            out_data.append(this_trip)

        return out_data

In [ ]:
# function to get a response from LLM

def get_response(prompt, model):

  completion = client.chat.completions.create(
      model=model,
      messages=[
          {
              "role": "user",
              "content": prompt
          }
      ]
  )

  response = completion.choices[0].message.content
  return response

# Data

In [ ]:
cnli_data = load_data("data/constructional_NLI/CxNLI_3_examples_test.tsv")
cnli_df = pd.DataFrame(columns=["construction","premise","hypothesis","relation"], data = cnli_data)
cnli_df.head(10)

,construction,premise,hypothesis,relation
0,let-alone,It is difficult enough for an individual to be...,"If an individual is consistent, a society migh...",1
1,let-alone,It is difficult enough for an individual to be...,It is easier for a society to be consistent th...,2
2,let-alone,It is difficult enough for an individual to be...,It is easier for an individual to be consisten...,0
3,let-alone,I would be distressed to hear of any ladies re...,If I would be distressed of a girl of your ten...,1
4,let-alone,I would be distressed to hear of any ladies re...,I would be equally distressed to hear of any l...,2
5,let-alone,I would be distressed to hear of any ladies re...,I would be more distressed to hear of a girl o...,0
6,let-alone,"None of these arguments is notably strong, let...","If one of these arguments were notably strong,...",1
7,let-alone,"None of these arguments is notably strong, let...",A conclusive argument is weaker than a notably...,2
8,let-alone,"None of these arguments is notably strong, let...",A conclusive argument is even stronger than a ...,0
9,let-alone,"Even so, such vague promises are no grounds fo...","If a promise is grounds for satisfaction, it i...",1


In [ ]:
cnli_df['construction'].unique()

array(['let-alone', 'way-manner', 'resultative', 'conative',
       'intransitive-motion', 'caused-motion', 'causative-with-CxN',
       'comparative-correlative'], dtype=object)

In [ ]:

for i in range (3):
  print("\nconstruction:", cnli_df['construction'][i])
  print("premise:",cnli_df['premise'][i])
  print("hypothesis:",cnli_df['hypothesis'][i])
  print("relation:",cnli_df['relation'][i])


construction: let-alone
premise: It is difficult enough for an individual to be consistent let alone a society.
hypothesis: If an individual is consistent, a society might also be consistent.
relation: 1

construction: let-alone
premise: It is difficult enough for an individual to be consistent let alone a society.
hypothesis: It is easier for a society to be consistent than an individual.
relation: 2

construction: let-alone
premise: It is difficult enough for an individual to be consistent let alone a society.
hypothesis: It is easier for an individual to be consistent than a society.
relation: 0


In [ ]:
challenge_cnli_data = load_data("data/challenge_constructional_NLI/challenge_CxNLI_test.tsv")
challenge_cnli_df = pd.DataFrame(columns=["construction","premise","hypothesis","relation"], data = challenge_cnli_data)
challenge_cnli_df.head(10)

,construction,premise,hypothesis,relation
0,resultative,A famous emperor buried China’s scholars alive...,China's scholars were fully alive before a fam...,0
1,resultative,A famous emperor buried China’s scholars alive...,Burying caused the scholars to become alive.,2
2,resultative,I bought the apples fresh.,The apples were completely fresh before I boug...,0
3,resultative,I bought the apples fresh.,Buying caused the apples to become fresh.,2
4,resultative,Any other bull would have drowned but U-bolt's...,U-bolt was fully alive before his energy kept ...,0
5,resultative,Any other bull would have drowned but U-bolt's...,U-bolt's energy caused him to become alive thr...,2
6,conative,I work at home.,I work away from home.,2
7,conative,The mother of one of the convicted gang member...,St. Patrick's Cathedral was the target of the ...,2
8,conative,He was assigned to teach at a school in the sm...,He taught in the opposite direction of the sch...,2
9,conative,He was assigned to teach at a school in the sm...,The school was the target of his teaching.,2


In [ ]:
for i in range (3):
  print("\nconstruction:", challenge_cnli_df['construction'][i])
  print("premise:",challenge_cnli_df['premise'][i])
  print("hypothesis:",challenge_cnli_df['hypothesis'][i])
  print("relation:",challenge_cnli_df['relation'][i])


construction: resultative
premise: A famous emperor buried China’s scholars alive with their books .
hypothesis: China's scholars were fully alive before a famous emperor buried them.
relation: 0

construction: resultative
premise: A famous emperor buried China’s scholars alive with their books .
hypothesis: Burying caused the scholars to become alive.
relation: 2

construction: resultative
premise: I bought the apples fresh.
hypothesis: The apples were completely fresh before I bought them.
relation: 0


# Activity

Using what was covered in the first notebook, prompt the model based on this new NLI task.

The following code blocks may be useful...

In [ ]:
prompt = "Hello World"

In [ ]:
print(prompt)

Hello World


In [ ]:
response = get_response(prompt, "gpt-4o-mini")

In [ ]:
print(response)

Hello! How can I assist you today?
